<a href="https://colab.research.google.com/github/edurso/obj-detect/blob/master/notebooks/tensorrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert TensroFlow `saved_model` to [TensorRT](https://github.com/NVIDIA/TensorRT/tree/main/samples/python/tensorflow_object_detection_api) Engine

define model location

In [ ]:
workdir = '/content/workspace/MyDrive/tensorflow/rapid_react/exported_models/trained_model/'
from google.colab import drive
import os
!mkdir -p /content/workspace
drive.mount('/content/workspace')

download `trained_model` from google drive to `workdir`

check python version

In [ ]:
!python3 --version

## Installation & Environment Configuration

install tfod api

In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models /content/models

if "utils" in pathlib.Path.cwd().parts:
  while "utils" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('utils').exists():
  !git clone --depth 1 https://github.com/edurso/obj-detect /content/utils

In [ ]:
%%bash

# update pip
python3 -m pip install --upgrade setuptools pip

# check opencv dependency versions
python3 -m pip uninstall opencv-python-headless==4.5.5.62 
python3 -m pip install opencv-python-headless==4.1.2.30
python3 -m pip install onnx-graphsurgeon==0.3.10 --index-url https://pypi.ngc.nvidia.com

In [ ]:
%%bash

# compile protos and install object_detection
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python3 -m pip install --use-feature=2020-resolver .

In [ ]:
!python3 -m pip show object-detection

install tensorrt

In [ ]:
!nvidia-smi
!python3 -m pip install nvidia-pyindex
!python3 -m pip install pycuda onnx==1.8.1 onnxruntime==1.8.0 tf2onnx==1.8.1
!python3 -m pip install onnx-graphsurgeon==0.3.10 --index-url https://pypi.ngc.nvidia.com
!python3 -m pip install --upgrade --index-url https://pypi.ngc.nvidia.com nvidia-tensorrt
import tensorrt
print(tensorrt.__version__)

get tensorrt utility files

In [ ]:
url = 'https://raw.githubusercontent.com/NVIDIA/TensorRT/main/samples/python/tensorflow_object_detection_api/{}.py'
files = ['build_engine', 'create_onnx', 'image_batcher', 'infer', 'compare_tf', 'infer', 'onnx_utils', 'visualize']
os.chdir('/content/utils/scripts')
for file in files:
    furl = url.format(file)
    if not os.path.exists('/content/utils/scripts/{}.py'.format(file)):
        !wget {furl}

## Create TensorRT Engine

In [ ]:
os.chdir('/content/utils/scripts')
!echo {workdir}saved_model
!python3 create_onnx.py \
    --pipeline_config {workdir}pipeline.config \
    --saved_model {workdir}saved_model \
    --onnx {workdir}model.onnx

In [ ]:
os.chdir('/content/utils/scripts')
!python3 build_engine.py \
    --onnx {workdir}model.onnx \
    --engine {workdir}engine.trt \
    --precision fp16 # for some reason this must be fp32, fp16 will not work

## Test TensorRT Engine

In [ ]:
os.chdir('/content/utils/scripts')
!python3 infer.py \
    --engine {workdir}engine.trt \
    --input /content/workspace/MyDrive/tensorflow/rapid_react/images/test \
    --output {workdir}output \
    --preprocessor fixed_shape_resizer \
    --labels {workdir}label_map.pbtxt

## Download TensorRT Engine (`engine.trt`)

In [ ]:
os.chdir('/content/utils/scripts')
from google.colab import files
files.download(workdir + 'engine.trt')

`engine.trt` is ready for use